In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import matplotlib
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from tqdm import tqdm
import os
import random
from typing import Tuple
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from torch.utils.data.dataset import TensorDataset, random_split
from torch.autograd import Variable
import numpy as np
import PIL
import pandas as pd
matplotlib.style.use('ggplot')
torch.cuda.empty_cache()

def set_seed(seed):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(1234)

In [2]:
class CvBlock(nn.Module):
    """(Conv2d => BN => ReLU) x 2"""

    def __init__(self, in_ch, out_ch):
        super(CvBlock, self).__init__()
        self.convblock = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.convblock(x)


class DownBlock(nn.Module):
    """Downscale + (Conv2d => BN => ReLU)*2"""

    def __init__(self, in_ch, out_ch):
        super(DownBlock, self).__init__()
        self.convblock = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            CvBlock(out_ch, out_ch),
        )

    def forward(self, x):
      
        return self.convblock(x)


class UpBlock(nn.Module):
    """(Conv2d => BN => ReLU)*2 + Upscale"""

    def __init__(self, in_ch, out_ch):
        super(UpBlock, self).__init__()
        self.convblock = nn.Sequential(
            CvBlock(in_ch, in_ch),
            nn.Conv2d(in_ch, out_ch * 4, kernel_size=3, padding=1),
            nn.PixelShuffle(2),
        )

    def forward(self, x):

        return self.convblock(x)


class Reshape(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.shape = args

    def forward(self, x):
        return x.view(self.shape)


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.encoder = nn.Sequential(
            DownBlock(1, 32),
            DownBlock(32, 64),
            DownBlock(64, 128),
            DownBlock(128, 256),
            nn.Flatten(),
        )
        self.z_mean = nn.Linear(4096, 256)
        self.z_var = nn.Linear(4096, 256)
        self.z_up = nn.Linear(256, 4096)
        self.decoder = nn.Sequential(
            nn.Linear(256,4096),
            Reshape(-1, 256, 4, 4),
            UpBlock(256, 128),
            UpBlock(128, 64),
            UpBlock(64, 32),
            UpBlock(32, 1),
        )

    def encode(self, x):
        h1 = self.encoder(x)
    
        return self.z_mean(h1), self.z_var(h1)

    @staticmethod
    def reparametrize(mu, logvar):
        std = logvar.mul(0.5).exp_()
        # if torch.cuda.is_available():
        #     eps = torch.cuda.FloatTensor(std.size()).normal_()
        # else:
        eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):

        h3 = self.decoder(z)
        return F.sigmoid(h3)

    def forward(self, x):
        mu, logvar = self.encode(x)
        # print(mu.shape, logvar.shape)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar, z

def conv3x3(in_planes: int,
            out_planes: int,
            stride: int = 1,
            groups: int = 1,
            dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes,
                     out_planes,
                     kernel_size=3,
                     stride=stride,
                     padding=dilation,
                     groups=groups,
                     bias=False,
                     dilation=dilation)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes,
                     out_planes,
                     kernel_size=1,
                     stride=stride,
                     bias=False)

In [3]:
class LinearAE(nn.Module):
    def __init__(self, in_size=16, latent_size=64):
        super(LinearAE, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(in_size, 64),
            nn.ReLU(),
            nn.Linear(64,latent_size),
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, 64),
            nn.ReLU(),
            nn.Linear(64, in_size),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        z = x
        x = self.decoder(x)

        return x, z

In [5]:
def sensor_train_dataloader(
    batch_size: int = 24,
    num_samples: int = 1,
    spread: float = 0.5,
    split: float = 0.8,
    seed: int = 1234,
):
    """
    Creates the dataset
    :param batch_size: the batch size
    :param num_samples: number of items per class/label/letter
    :param spread: the std for normal sampling
    :param split: train-val split (<1). Number given is allotted as the train %
    :param seed: seed for the "random" dataset generator
    :return: the dataloaders
    """
    np.random.seed(seed)
    ideal_sensory_values = np.random.randint(-24, 23, (24, 16))
    

    classes = 24

    dataset     = list()
    dataset_cor = list()

    for letter in range(classes):
        for _ in range(num_samples):
            sensors = []
            un_cor = []
            for sensor in ideal_sensory_values[letter]:
                r = np.random.normal(loc=sensor, scale=spread)
                sensors.append(r)
                un_cor.append(r)
            
            sensors = np.array(sensors)
            un_cor = np.array(un_cor)
            dataset.append([un_cor, np.array([letter])])
    

            if np.random.choice([True,False],p=[0.7,0.3]):
                indices = np.random.choice(np.arange(sensors.size), replace=False, size=int(sensors.size * 0.3))
                sensors[indices] = np.random.choice([-100,100])
                
            dataset_cor.append([sensors, np.array([letter])])
            
    
    x = list()
    w = list()
    y = list()
    
    for i in range(num_samples * classes): #24 if we train fully instead of classes
        x.append(dataset_cor[i][0])
        w.append(dataset[i][0])
        y.append(dataset_cor[i][1])

    x = np.array(x)
    w = np.array(w)
    y = np.array(y)

    tensor_x = torch.Tensor(x)
    tensor_w = torch.Tensor(w)
    tensor_y = torch.Tensor(y)

    tensor_dataset = TensorDataset(tensor_x, tensor_w, tensor_y)
    train_loader = DataLoader(tensor_dataset, batch_size=batch_size, shuffle=True)

    return train_loader

In [6]:
class Dataset(object):
    
    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])

class DatasetMNIST(Dataset):
    
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path)
        self.data = self.data.sort_values('label')
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):

        image = self.data.iloc[index, 1:].values.astype(np.uint8).reshape((28, 28, 1))
        label = self.data.iloc[index, 0]

        if (label > 9):
          label = label - 1
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

def image_dataloader(batch_size = 24):

    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((64,64)),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
    ])


    train_data = DatasetMNIST('/home/ubuntu/Latent-Transfer/DomainB/DomainB_dataset/sign_mnist_1s26c.csv', transform = transform)


    train_loader = DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=False
    )



    return train_loader

In [8]:
img_trainloader= image_dataloader(batch_size = 1)
len(img_trainloader)

24

In [9]:
sensor_trainloader = sensor_train_dataloader(batch_size = 1)
len(sensor_trainloader)

24

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sensor_model = LinearAE(16, 256)
sensor_model.load_state_dict(torch.load('/home/ubuntu/Latent-Transfer/DomainA/models/domainA-encoder-complete.pt'))
sensor_model.eval()

image_model = VAE()
image_model.load_state_dict(torch.load('/home/ubuntu/Latent-Transfer/DomainB/outputs/models/cnn_vae_151.pth'))
image_model.eval()

VAE(
  (encoder): Sequential(
    (0): DownBlock(
      (convblock): Sequential(
        (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): CvBlock(
          (convblock): Sequential(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
            (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (5): ReLU(inplace=True)
          )
        )
      )
    )
    (1): DownBlock(
      (convblock): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [17]:
def get_min_and_max(dataloader, device) -> Tuple[float, float]:
    """
    Finds the min and max of the dataset.
    This is used for Normalization of the dataset.

    :param dataloader: dataloader to calculate it for
    :param device: device to run computations on
    :return: tuple of mean and std
    """
    min_val, max_val = torch.Tensor([999]).to(device), torch.Tensor([-999]).to(device)
    for data, data_uncor, _ in tqdm(dataloader):
        data_uncor = data_uncor.to(device)
        min_val = torch.min(min_val, torch.min(data_uncor))
        max_val = torch.max(max_val, torch.max(data_uncor))

    return min_val.item(), max_val.item()

In [18]:
def extract_latent(sensor_trainloader ,
                   img_trainloader,  
                   min_val_train, 
                   max_val_train, 
                   ):

    img_loader = [img_trainloader]
    sensor_loader = [sensor_trainloader]

    for i in img_loader:
        recon_img = []
        recon_img_label = []
        for batch_idx, data in enumerate(tqdm(i)):
        
            img, label = data
            img = Variable(img)

            recon_batch, mu, logvar, z = image_model(img)

            recon_img.append(z.detach().numpy())
            recon_img_label.append(label.detach().numpy())

        ri = np.asarray(recon_img)
        rl = np.asarray(recon_img_label)

        np.save('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces/latent_img_overfit', ri)
        np.save('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces/latent_img_label_overfit', rl)
      
    
     
    for i in sensor_loader:
        recon_sensor = []
        recon_sensor_corr = []
        recon_sensor_label = []
        for batch_idx, data in enumerate(i):

            vector, vector_uncor, label = data

            
            min_val = min_val_train
            max_val = max_val_train
            

            vector     = vector
            vector     = (vector - min_val) / (max_val - min_val )
            vector_uncor = vector_uncor
            vector_uncor = (vector_uncor - min_val) / (max_val - min_val )
            
            y, z = sensor_model(vector)
            y_u, z_u = sensor_model(vector_uncor)

            recon_sensor_corr.append(z.detach().numpy())
            recon_sensor.append(z_u.detach().numpy())
            recon_sensor_label.append(label.detach().numpy())    
        
        si = np.asarray(recon_sensor_corr)
        su = np.asarray(recon_sensor)
        sl = np.asarray(recon_sensor_label)
        

       

        np.save('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces/latent_sensor_corrupted_overfit', si)
        np.save('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces/latent_sensor_uncorrupted_overfit', su)
        np.save('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces/latent_sensor_label_overfit', sl)
       

In [19]:
min_val_train, max_val_train = get_min_and_max(sensor_trainloader, device)

100%|██████████| 24/24 [00:00<00:00, 3457.56it/s]


In [20]:
extract_latent(sensor_trainloader, 
               img_trainloader,                              
               min_val_train, 
               max_val_train, 
               )

  0%|          | 0/24 [00:00<?, ?it/s]/home/ubuntu/miniconda3/envs/latent/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 24/24 [00:01<00:00, 16.61it/s]


In [21]:
img = np.load('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces_overfit/latent_img_overfit.npy')
sensor = np.load('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces_overfit/latent_sensor_uncorrupted_overfit.npy')

In [22]:
img.shape, sensor.shape

((24, 1, 256), (24, 1, 256))

In [33]:
img[0][:,:10], sensor[0][:,:10]

(array([[ 0.8001833 , -0.2443443 ,  2.0559378 ,  0.77357346, -1.96828   ,
         -0.6645336 , -0.56715226, -0.4280931 , -0.770191  , -0.98206365]],
       dtype=float32),
 array([[-0.30570972,  0.19028081,  0.10979673, -0.0039343 , -0.08206388,
         -0.05025379, -0.07782561, -0.17995879, -0.2827127 , -0.5074086 ]],
       dtype=float32))

In [34]:
img[1][:,:10], sensor[1][:,:10]

(array([[-0.02945861, -0.32403868,  0.8381888 , -1.3386047 , -0.62591594,
         -0.00530443,  0.8494981 ,  0.69293916, -0.9786075 , -0.47164962]],
       dtype=float32),
 array([[ 0.16656825, -0.19828467,  0.29497063, -0.1966173 ,  0.25626114,
          0.21926247,  0.14281817,  0.16365409, -0.34524703, -0.16361883]],
       dtype=float32))

In [35]:
img[23][:,:10], sensor[23][:,:10]

(array([[ 1.9565477 ,  0.9846157 , -1.418498  , -0.48450425,  1.4652798 ,
          0.37195742, -0.08091757, -0.46949401,  1.2331209 , -0.22979861]],
       dtype=float32),
 array([[-0.02898763, -0.67587006,  0.19560233, -0.15385172,  0.05916131,
          0.29706925,  0.5604278 ,  0.32146385,  0.32568794, -0.23067227]],
       dtype=float32))